# Convert delta output to Pandas dataframes

In [1]:
import pathlib
import numpy as np
import pandas as pd
from delta_postprocess import delta_to_df 

import os
import shutil


In [2]:
#set paths
root = pathlib.Path(pathlib.Path.home(), 'home', 'Delta2_Caulobacter')
data_dir = root / 'processed_dataPA'
out_dir = root / 'outputPA'
out_dir.mkdir(exist_ok=True)

csv_dir = out_dir / 'csv_files'
csv_dir.mkdir(exist_ok=True)

movie_dir = out_dir / 'mp4_files'
movie_dir.mkdir(exist_ok=True)

#find subfolders
folder_names = [f.name for f in sorted(data_dir.glob('UJP*'))]
print(folder_names)

['AKS1088', 'AKS1089', 'AKS1090', 'AKS1091', 'AKS1092', 'AKS1093', 'AKS1094', 'AKS1095', 'AKS1096', 'AKS1097', 'AKS1098', 'AKS1099', 'AKS1101', 'AKS1102', 'AKS1103', 'AKS1104', 'AKS1105', 'AKS1106']


In [23]:
#copy movies 
for folder in folder_names:
    #get images in subfolder
    movie_names = [f.name for f in sorted((data_dir / folder).glob('*TL*'))]

    for idx, movie in enumerate(movie_names):        
        #path to current position        
        datafiles = [f.name for f in sorted((data_dir / folder / movie).glob('*.mp4'))]
        mp4_name = '%s_%s.mp4' %(folder,movie)
        
        path_old = data_dir / folder / movie / datafiles[0]
        path_new = movie_dir / mp4_name
        
        shutil.copyfile(path_old, path_new)

In [ ]:
df_list = []
for folder in folder_names:
    #get images in subfolder
    movie_names = [f.name for f in sorted((data_dir / folder).glob('*TL*'))]


    for idx, movie in enumerate(movie_names):        
        #path to current position        
        datafiles = [f.name for f in sorted((data_dir / folder / movie).glob('*.pkl'))]
        short_name = '%s_%s' %(folder,movie)
                
        df = delta_to_df(data_dir / folder / movie / datafiles[0])
        
        df['strain'] = folder
        df['movie_name'] = short_name
        df['replicate'] = idx
        
        #save data-frame
        save_name = short_name + '.csv'
        df.to_csv(csv_dir / save_name)

        df_list.append(df)

In [25]:
df_combined = pd.concat(df_list, ignore_index=True)

In [26]:
save_name = 'pa_combined_data.csv'
df_combined.to_csv(out_dir / save_name)

In [27]:
df_combined.head()

,id_seg,id_cell,id_par,id_colony,id_d1,id_d2,id_sib,frames,new_pole,old_pole,...,width,area,perimeter,fluo1,fluo2,x_pos,y_pos,strain,movie_name,replicate
0,0,0,-1,0,-1,-1,-1,0,"[4, 706]","[2, 706]",...,7.0,32.5,11,470.422222,411.577778,705.777778,2.955556,AKS1088,AKS1088_TL01_01,0
1,0,0,-1,0,-1,-1,-1,1,"[4, 700]","[3, 700]",...,6.0,31.5,12,473.651163,400.348837,700.255814,2.813953,AKS1088,AKS1088_TL01_01,0
2,0,0,-1,0,-1,-1,-1,2,"[3, 705]","[0, 699]",...,6.0,26.5,10,458.216216,372.648649,702.243243,2.405405,AKS1088,AKS1088_TL01_01,0
3,0,0,-1,0,-1,-1,-1,3,"[2, 704]","[2, 703]",...,6.0,23.0,8,453.787879,371.757576,703.606061,2.242424,AKS1088,AKS1088_TL01_01,0
4,0,0,-1,0,-1,-1,-1,4,"[3, 704]","[2, 704]",...,6.0,30.5,9,473.738095,368.166667,704.047619,2.642857,AKS1088,AKS1088_TL01_01,0


In [28]:
df_combined.shape

(587458, 22)